In [1]:
!pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.9 MB/s eta 0:00:00


In [2]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

In [3]:
import numpy as np
import time
import psutil
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
X_train = pd.read_csv('X_train_classification.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train_classification.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test_classification.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test_classification.csv')
y_test=y_test.to_numpy().reshape(-1)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [6]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [7]:
# def initialize_parameters(n_features):
#     w = np.zeros(n_features)
#     b = 0
#     return w, b

weights_enc= np.zeros(X_train.shape[1])
bias_enc=0
lr=0.01
C=0.01
C2=ts.ckks_vector(context,[C])
n_samples, n_features = X_train.shape

In [8]:
n_samples, n_features

(80, 2)

In [9]:
weights_enc=ts.ckks_vector(context,weights_enc)

In [10]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [11]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [12]:
# def sgd_svm(X, y, learning_rate=0.01, C=1.0, weights_enc, bias_enc,n_samples, n_features, epochs=3):

#     for epoch in range(epochs):
#         for i in range(n_samples):
#             xi = X[i]
#             yi = y[i]
#             margins = 1 - yi * (np.dot(xi, w) + b)
#             if margins > 0:
#                 w = w - learning_rate * (w - C * yi * xi)
#                 b = b - learning_rate * (-C * yi)
#             else:
#                 w = w - learning_rate * w

#     return w, b

In [13]:
xi=enc_X_train[0]
yi=enc_y_train[0]

xi,yi

(<tenseal.tensors.ckksvector.CKKSVector at 0x7c929f9cbb80>,
 <tenseal.tensors.ckksvector.CKKSVector at 0x7c92e457e7a0>)

In [14]:
margins = 1 - yi * (xi.dot(weights_enc)+bias_enc)
margins.decrypt()

[1.000000040854716]

In [15]:
if margins.decrypt()[0] > 0:
  w=C*yi
  w=relinearlizing_scalar(w)
  y=w*xi
  # y=relinearlizing_vector(y)
  x=weights_enc-y
  x=relinearlizing_vector(x)

  weights_enc=weights_enc-lr*x
  bias_enc=bias_enc-lr*(-C*yi)
  weights_enc=relinearlizing_vector(weights_enc)
  bias_enc=relinearlizing_scalar(bias_enc)
else:
  weights_enc=weights_enc-lr*weights_enc
  weights_enc=relinearlizing_vector(weights_enc)



In [16]:
weights_enc.decrypt()

[4.909826816849461e-05, 5.9499046206301325e-05]

In [17]:
bias_enc.decrypt()

[9.999907024406289e-05]

In [18]:
xi.decrypt()

[0.4909095485986323, 0.595138859239811]

In [19]:
yi.decrypt()

[1.0000000006190544]

In [20]:
xi=enc_X_train[1]
yi=enc_y_train[1]

margins = 1 - yi * (xi.dot(weights_enc)+bias_enc)
if margins.decrypt()[0] > 0:
  w=C*yi
  w=relinearlizing_scalar(w)
  y=w*xi
  # y=relinearlizing_vector(y)
  x=weights_enc-y
  x=relinearlizing_vector(x)

  weights_enc=weights_enc-lr*x
  bias_enc=bias_enc-lr*(-C*yi)
  weights_enc=relinearlizing_vector(weights_enc)
  bias_enc=relinearlizing_scalar(bias_enc)
else:
  weights_enc=weights_enc-lr*weights_enc
  weights_enc=relinearlizing_vector(weights_enc)

In [21]:
weights_enc.decrypt()

[-0.00010959892794501803, 0.00022620021990180035]

In [22]:
bias_enc.decrypt()

[-6.638697472067613e-11]

In [23]:
xi=enc_X_train[2]
yi=enc_y_train[2]

margins = 1 - yi * (xi.dot(weights_enc)+bias_enc)
if margins.decrypt()[0] > 0:
  w=C*yi
  w=relinearlizing_scalar(w)
  y=w*xi
  # y=relinearlizing_vector(y)
  x=weights_enc-y
  x=relinearlizing_vector(x)

  weights_enc=weights_enc-lr*x
  bias_enc=bias_enc-lr*(-C*yi)
  weights_enc=relinearlizing_vector(weights_enc)
  bias_enc=relinearlizing_scalar(bias_enc)
else:
  weights_enc=weights_enc-lr*weights_enc
  weights_enc=relinearlizing_vector(weights_enc)

In [24]:
weights_enc.decrypt()

[-0.00011059883947814214, 0.0001992992238379988]

In [25]:
bias_enc.decrypt()

[9.999930308041222e-05]

In [26]:
# # def predict(X, w, b):
# #     return np.sign(np.dot(X, w) + b)

# def predict(X_test,weights_enc,bias_enc):
#   y_pred=[]
#   for xi in X_test:
#     rog=xi.dot(weights_enc)+bias_enc
#     hell=rog.decrypt()[0]
#     if hell>0:
#       y_pred.append(1)
#     else:
#       y_pred.append(-1)
#   return y_pred

In [27]:
# enc_X_test=[]
# for i in X_test:
#   p=ts.ckks_vector(context,i)
#   enc_X_test.append(p)

In [28]:
# y_pred=predict(enc_X_test,weights_enc,bias_enc)

In [29]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test,y_pred)